In [105]:
# !pip install webdriver_manager
# !pip install pandas
# !pip install selenium
# !pip install pillow

In [16]:
# import
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import requests
import csv
import os
from PIL import Image
from io import BytesIO

In [22]:
# departments = ["가정의학과", "간담췌외과", "감염내과", "갑상선내분비외과", "건강증진의학과", "내분비내과", "대장항문외과", "류마티스내과", "마취통증의학과", "방사선종양학과", "병리과", "비뇨의학과", "산부인과", "성형외과", "소아외과", "소아청소년과", "소화기내과", "순환기내과", "신경외과", "신장내과", "심장혈관흉부외과"]
# 신경과 누락
# 임상약리과 의료진 없음으로 제외

departments = [ "안과", "알레르기내과", "영상의학과", "완화의학과", "위장관외과", "유방외과", "응급의학과", "이비인후과", "입원의학과", "재활의학과", "정신건강의학과", "정형외과", "종양내과", "중환자외상외과", "직업환경의학과", "진단검사의학과", "치과병원", "피부과", "핵의학과", "혈관ㆍ이식외과", "혈액내과", "호흡기내과"]


all_doctor_info_list = []
    

for department in departments:
    
    doctor_info_list = []
    
    # 1. chrome driver 띄우기
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--ssl-version-max=tls1.2")
    chrome_options.add_experimental_option("detach", True)

    # 가톨릭대학교 서울성모병원 url
    url = 'https://www.cmcseoul.or.kr/page/department/A'
    driver = webdriver.Chrome(options=chrome_options)
    driver.implicitly_wait(10)
    driver.get(url)
    
    # 1. 과 검색

    # 1.1. 검색 버튼 클릭
    search_button = driver.find_element(By.CLASS_NAME, 'btn_search')
    search_button.click()
    
    # 1.2. 검색창 버튼 클릭
    search_box = driver.find_element(By.ID, 'kwd')
    time.sleep(3)
    search_box.click()
    
    # 1.3. 진료과 이름 입력
    search_box.clear() 
    search_box.send_keys(department)
    
    # 1.4. 검색 버튼 클릭
    search_button2 = driver.find_element(By.CLASS_NAME, 'search_btn')
    search_button2.click()

    # 1.5. 의료진 버튼 클릭
    docker_link = driver.find_element(By.XPATH, '//a[contains(@href, "javascript:goCategory(\'DOCTOR\');")]')
    docker_link.click() 

    # 2. 의료진 정보 가져오기

    # 2.1. html 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    soup

    # 3.2. 의료진 정보 가져오기

    doctor_tags = soup.find_all('div', class_='doc_list_wrap')
    clinic_tags = soup.find_all('dt', string='전문분야')

    for i in range(1, len(doctor_tags)):        
        if doctor_tags[i]:
            dptm_name_elem = doctor_tags[i].find('strong').text
            name_elem, dptm_elem = dptm_name_elem.split('[')
            img_elem = doctor_tags[i].find('img')['src']
            position_elem = doctor_tags[i].find('img')['alt']
            clinic_elem = clinic_tags[i].find_next('p').text
            
            if dptm_name_elem and name_elem and dptm_elem and img_elem and position_elem and clinic_elem:
                
                dptm = dptm_elem.strip('[]')
                name = name_elem.strip()
                img_name = "가톨릭대학교" + " " + "서울성모병원" + " " + department + " " + name + ".png"
                position = position_elem.replace(name, '').split(' ')[0]
                clinic = clinic_elem
                img_src = "https://www.cmcseoul.or.kr" + img_elem

                # # 3.3. 이미지 다운로드
                # response = requests.get(img_src, verify=False)
                # if response.status_code == 200:
                #     image_data = response.content
                #     image = Image.open(BytesIO(image_data))
                #     image.save(f"03/images/{img_name}")  

                # 3.4. 의료진 데이터 저장
                doctor_info = {
                    'dptm': dptm,
                    'name': name,
                    'img_name': img_name, 
                    'position': position,
                    'clinic': clinic,
                }
                doctor_info_list.append(doctor_info)
                
                all_doctor_info_list.extend(doctor_info_list)
                
                   
    driver.quit()

# 4. csv로 내보내기
with open("03/doctor_info.csv", 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['dptm', 'name', 'img_name', 'position', 'clinic']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for doctor_info in all_doctor_info_list:
        writer.writerow(doctor_info)